## 0. Simulate dataset (*)
# Simulate datasets according to these rules:
* ### set random seed to 42
* ### (1000,2) samples from  𝑋∼$\mathcal{U}$(0,1)  , i.e. 1000 rows, 2 columns.
* ### 1000 samples from  $\epsilon \sim N$(0,1) 
* ### 𝑦=3𝑥1+5𝑥2+3+𝜖  , where  𝑥𝑖  is column  𝑖  of  𝑋 
* ### Finally add a column of ones for the intercept to  𝑋 .

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)
samples = 1000

X = np.random.rand(samples, 2)
Epsilon = np.random.normal(0, 1, size=(samples, 2)) # mean 0, std 1

print(Epsilon.shape, X.shape)


Epsilon, X

y = X[:3]+



(1000, 2) (1000, 2)


(array([[-0.87798259, -0.82688035],
        [-0.22647889,  0.36736551],
        [ 0.91358463, -0.80317895],
        ...,
        [ 0.6870658 ,  1.46654086],
        [-1.11158025, -0.03582558],
        [-0.53145455, -1.56785943]]),
 array([[0.37454012, 0.95071431],
        [0.73199394, 0.59865848],
        [0.15601864, 0.15599452],
        ...,
        [0.75137509, 0.65695516],
        [0.95661462, 0.06895802],
        [0.05705472, 0.28218707]]))